In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.cluster import HDBSCAN
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2

In [2]:
def cluster_image(image_path, n_clusters):
    image = Image.open(image_path)

    image_np = np.array(image)

    lab = cv2.cvtColor(image_np, cv2.COLOR_RGB2LAB)
    pixels = lab.reshape(-1, 3)

    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(pixels)

    labels = kmeans.labels_.reshape(image_np.shape[0], image_np.shape[1])

    # Visualiza usando os centros em RGB
    centers_lab = kmeans.cluster_centers_.astype("uint8")
    centers_rgb = cv2.cvtColor(centers_lab[None, :, :], cv2.COLOR_LAB2RGB)[0]

    segmented = centers_rgb[labels]
    clustered_image = Image.fromarray(segmented)
    
    return kmeans, clustered_image

In [3]:
c = cluster_image("images/241216_antifonario_folha1_frente_200_RGB.tif", n_clusters=3)
c[1].save("clustered_doc.png")

In [4]:
img = Image.open("images/241216_antifonario_folha1_frente_200_RGB.tif")
img_np = np.array(img)
lab = cv2.cvtColor(img_np, cv2.COLOR_RGB2LAB)
X_new = lab.reshape(-1,3)


labels = c[0].predict(X_new)
labels2 = labels.reshape(img_np.shape[0], img_np.shape[1])
labels = labels2

In [5]:
n_clusters = 3
mask = [(labels == i).astype(np.uint8) * 255 for i in range(n_clusters)]

# criar imagens segmentadas direto com numpy
for i in range(n_clusters):
    # cria máscara 3D (mesmas dimensões da imagem)
    mask3 = mask[i][:, :, None]  # adiciona canal
    cluster_img = np.where(mask3 == 255, img_np, 0).astype(np.uint8)

    Image.fromarray(cluster_img).save(f"cluster3_{i}.png")

In [ ]:
clusterer = HDBSCAN(min_cluster_size=10).fit(X_new)
cluster_labels = clusterer.fit_predict(X_new)
print(f"fez cluster com {len(set(cluster_labels))} clusters")
labels = cluster_labels.reshape(img_np.shape[0], img_np.shape[1])
n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
mask = [(labels == i).astype(np.uint8) * 255 for i in range (n_clusters)]


In [ ]:
# criar imagens segmentadas direto com numpy
for i in range(n_clusters):
    # cria máscara 3D (mesmas dimensões da imagem)
    mask3 = mask[i][:, :, None]  # adiciona canal
    cluster_img = np.where(mask3 == 255, img_np, 0).astype(np.uint8)

    Image.fromarray(cluster_img).save(f"cluster4_{i}.png")